### Задание на ПР-7:
В приложенном датасете приведены данные о реальных больных сердечно-сосудистыми заболеваниями. Последняя компонента в каждом экземпляре - бинарная метка (0 - нет болезни, 1 - есть болезнь). Ваша задача - построить классификатор, который на данных нового пациента предскажет наличие у него болезни. Затем нужно вычислить метрики качества вашего классификатора. 
1. Сделайте препроцессинг.
2. Разделите данные на обучающие и тестовые (70/30), причем проследите чтобы пропорции 0 и 1 в них были примерно одинаковыми.
3. Напишите функцию, которая вычислит коэффициенты логистической регрессии на обучающей выборке.
4. Напишите функцию, которая предскажет метку на тестовом экземпляре данных, пользуясь вычисленными коэффициентами логистической регрессии (ЛР).
5. Выберите из датасета такие экземпляры, на которых логистическая регрессия дает нестабильные предсказания. Таких экземпляров должно быть не менее 20% от всего датасета. Отметьте их третьим типом метки (например, -1, или 2 - как вам нравится больше).
6. Постройте классификацию (выполните пп. 3 и 4) методом ближайших соседей (БС). 
7. Измерьте качество классификации для ЛР и БС по метрикам Precision, Recall, F1 (для бинарной классификации) и Accuracy (для тернарной классификации). 
8. Интерпретируйте результаты - сделайте выводы о качестве классификации, проведенной на основе разных методов и с разным числом классов.

#### ЗАДАНИЕ 1: ПРЕПРОЦЕССИНГ ДАННЫХ

In [75]:
# 1. Сделайте препроцессинг.
# Импорт библиотек
import math  # для математических операций
import random  # для случайных чисел и перемешивания
import csv  # для работы с CSV файлами

def load_heart_data(filename):
    """Загружает данные о сердечных заболеваниях из CSV файла"""
    dataset = []  # список для хранения данных
    with open(filename, 'r') as file:  # открываем файл для чтения
        csv_reader = csv.reader(file)  # создаем CSV reader
        headers = next(csv_reader)  # читаем и сохраняем заголовки
        for row in csv_reader:  # читаем каждую строку
            dataset.append(row)  # добавляем строку в датасет
    return dataset  # возвращаем загруженные данные

def preprocess_heart_data(raw_data):
    """Преобразует категориальные признаки в числовые и масштабирует данные"""
    # Словари для преобразования категориальных признаков
    sex_map = {'M': 1, 'F': 0}  # мужской=1, женский=0
    chest_pain_map = {'ATA': 0, 'NAP': 1, 'ASY': 2, 'TA': 3}  # типы боли в груди
    ecg_map = {'Normal': 0, 'ST': 1, 'LVH': 2}  # типы ЭКГ
    angina_map = {'N': 0, 'Y': 1}  # наличие стенокардии
    slope_map = {'Up': 0, 'Flat': 1, 'Down': 2}  # наклон ST сегмента
    
    processed_data = []  # список для обработанных данных
    
    for patient in raw_data:  # обрабатываем каждого пациента
        processed_patient = []  # список для обработанного пациента
        
        # Преобразуем каждый признак в числовой формат
        processed_patient.append(int(patient[0]))  # возраст -> целое число
        processed_patient.append(sex_map[patient[1]])  # пол по словарю
        processed_patient.append(chest_pain_map[patient[2]])  # тип боли
        processed_patient.append(int(patient[3]))  # давление -> целое число
        processed_patient.append(int(patient[4]))  # холестерин -> целое число
        processed_patient.append(int(patient[5]))  # уровень сахара -> целое число
        processed_patient.append(ecg_map[patient[6]])  # ЭКГ по словарю
        processed_patient.append(int(patient[7]))  # пульс -> целое число
        processed_patient.append(angina_map[patient[8]])  # стенокардия по словарю
        processed_patient.append(float(patient[9]))  # ST депрессия -> дробное число
        processed_patient.append(slope_map[patient[10]])  # наклон ST по словарю
        processed_patient.append(int(patient[11]))  # метка болезни -> целое число
        
        processed_data.append(processed_patient)  # добавляем обработанного пациента
    
    # Масштабируем признаки (кроме метки)
    # Стандартизация: x_std = (x - mean) / std
    num_features = len(processed_data[0]) - 1  # количество признаков (без метки)
    for feature_idx in range(num_features):  # для каждого признака
        feature_values = []  # создаем пустой список для значений признака
        for patient in processed_data:  # для каждого пациента в обработанных данных
            value = patient[feature_idx]  # берем значение признака по индексу
            feature_values.append(value)  # добавляем значение в список
        
        # Вычисляем среднее и стандартное отклонение
        mean_val = sum(feature_values) / len(feature_values)  # среднее значение
        
        # Вычисляем сумму квадратов отклонений
        sum_squared_diff = 0.0
        for x in feature_values:
            squared_diff = (x - mean_val) ** 2 # (x - μ)²
            sum_squared_diff += squared_diff

        # Вычисляем стандартное отклонение
        variance = sum_squared_diff / len(feature_values)  # дисперсия
        std_val = variance ** 0.5  # квадратный корень = стандартное отклонение
        
        if std_val > 0:  # если значения не постоянны
            for i in range(len(processed_data)):  # масштабируем каждый элемент
                processed_data[i][feature_idx] = (processed_data[i][feature_idx] - mean_val) / std_val
    
    return processed_data  # возвращаем обработанные данные

# Загружаем и обрабатываем данные
raw_heart_data = load_heart_data('heart.csv')  # загружаем исходные данные
print("Первые 5 строк исходных данных:")
for i in range(min(5, len(raw_heart_data))):  # выводим первые 5 строк
    print(f"Пациент {i+1}: {raw_heart_data[i]}")

processed_heart_data = preprocess_heart_data(raw_heart_data)  # обрабатываем данные
print(f"\nЗагружено {len(processed_heart_data)} пациентов")  # выводим количество пациентов
print(f"Количество признаков: {len(processed_heart_data[0]) - 1}")  # выводим количество признаков

Первые 5 строк исходных данных:
Пациент 1: ['40', 'M', 'ATA', '140', '289', '0', 'Normal', '172', 'N', '0', 'Up', '0']
Пациент 2: ['49', 'F', 'NAP', '160', '180', '0', 'Normal', '156', 'N', '1', 'Flat', '1']
Пациент 3: ['37', 'M', 'ATA', '130', '283', '0', 'ST', '98', 'N', '0', 'Up', '0']
Пациент 4: ['48', 'F', 'ASY', '138', '214', '0', 'Normal', '108', 'Y', '1.5', 'Flat', '1']
Пациент 5: ['54', 'M', 'NAP', '150', '195', '0', 'Normal', '122', 'N', '0', 'Up', '0']

Загружено 918 пациентов
Количество признаков: 11


#### ЗАДАНИЕ 2: РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ТЕСТОВУЮ ВЫБОРКИ

In [76]:
# 2. Разделите данные на обучающие и тестовые (70/30), причем проследите чтобы пропорции 0 и 1 в них были примерно одинаковыми.
def split_data_stratified(dataset, test_ratio=0.3, label_index=-1):
    """Разделяет данные на обучающую и тестовую выборки с сохранением пропорций классов"""
    # Разделяем данные по классам
    class_0 = []  # пациенты без болезни
    class_1 = []  # пациенты с болезнью
    
    for patient in dataset:  # распределяем пациентов по классам
        if patient[label_index] == 0:  # если пациент без болезни
            class_0.append(patient)  # добавляем в класс 0
        else:  # если пациент с болезнью
            class_1.append(patient)  # добавляем в класс 1
    
    # Перемешиваем данные в каждом классе
    random.shuffle(class_0)  # перемешиваем класс 0
    random.shuffle(class_1)  # перемешиваем класс 1
    
    # Вычисляем размер тестовой выборки для каждого класса
    test_size_0 = int(len(class_0) * test_ratio)  # размер теста для класса 0 , 30% от class_0
    test_size_1 = int(len(class_1) * test_ratio)  # размер теста для класса 1 , 30% от class_1
    
    # Формируем тестовую выборку
    test_data = class_0[:test_size_0] + class_1[:test_size_1]  # берем первые элементы для теста
    
    # Формируем обучающую выборку
    train_data = class_0[test_size_0:] + class_1[test_size_1:]  # остальные для обучения
    
    # Перемешиваем итоговые выборки
    random.shuffle(train_data)  # перемешиваем обучающую выборку
    random.shuffle(test_data)  # перемешиваем тестовую выборку
    
    # Разделяем на признаки и метки
    X_train = [] # признаки
    y_train = [] # метки
    for patient in train_data:  # для каждого пациента в обучающей выборке
        X_train.append(patient[:-1])  # добавляем признаки (все кроме последнего)
        y_train.append(patient[-1])   # добавляем метку (последний элемент)
    
    X_test = [] # признаки
    y_test = [] # метки
    for patient in test_data:  # для каждого пациента в тестовой выборке
        X_test.append(patient[:-1])  # добавляем признаки (все кроме последнего)
        y_test.append(patient[-1])   # добавляем метку (последний элемент)
    
    return X_train, X_test, y_train, y_test  # возвращаем разделенные данные

# Разделяем данные
X_train, X_test, y_train, y_test = split_data_stratified(processed_heart_data)  # разделяем данные

print(f"Обучающая выборка: {len(X_train)} пациентов")  # выводим размер обучающей выборки
print(f"Тестовая выборка: {len(X_test)} пациентов")  # выводим размер тестовой выборки

# Подсчитываем распределение классов в обучающей выборке
class_count_train = [0, 0]  # счетчики для классов 0 и 1
for label in y_train:  # проходим по всем меткам обучения
    class_count_train[label] += 1  # увеличиваем соответствующий счетчик

print(f"Распределение классов в обучении: 0={class_count_train[0]}, 1={class_count_train[1]}")  # выводим распределение

Обучающая выборка: 643 пациентов
Тестовая выборка: 275 пациентов
Распределение классов в обучении: 0=287, 1=356


#### ЗАДАНИЕ 3: РЕАЛИЗАЦИЯ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ И ОБУЧЕНИЕ МОДЕЛИ

**Для логистической регрессии градиент вычисляется по формуле:**

∂L/∂α = (ŷ - y) - градиент для свободного члена

∂L/∂βᵢ = (ŷ - y) × xᵢ - градиент для коэффициента признака i

In [77]:
def sigmoid_function(x):
    """Сигмоидная функция с защитой от переполнения"""
    if x < -700:  # для очень маленьких значений, x стремится в -беск
        return 0.0  # возвращаем 0
    elif x > 700:  # для очень больших значений, x стремится в +беск
        return 1.0  # возвращаем 1
    return 1.0 / (1.0 + math.exp(-x))  # вычисляем сигмоиду

def compute_logistic_prediction(features, weights):
    """Вычисляет предсказание логистической регрессии"""
    # Промежуточные вычисления z = α + β₁x₁ + β₂x₂ + ... + βₙxₙ
    z = weights[0]  # начинаем со свободного члена α
    for i in range(len(features)):  # для каждого признака
        z += weights[i + 1] * features[i]  # добавляем взвешенный признак, то есть β₁x₁ + β₂x₂ + ...
    return sigmoid_function(z)  # возвращаем ŷ (предсказание модели) через сигмоиду

def compute_log_loss(features, true_label, weights):
    """Вычисляет логарифмическую потерю для одного наблюдения"""
    prediction = compute_logistic_prediction(features, weights)  # получаем предсказание, вероятность того, что пациент болен
    epsilon = 1e-15  # маленькое число для избежания log(0)
    prediction = max(min(prediction, 1 - epsilon), epsilon)  # ограничиваем предсказание

    # Минимизируем ошибку, поэтому минус
    if true_label == 1:  # если истинная метка = 1 (пациент болен)
        return -math.log(prediction)  # -log(ŷ)
    else:  # если истинная метка = 0 (паицент здоров)
        return -math.log(1 - prediction)  # -log(1-ŷ)

def compute_gradient(features, true_label, weights):
    """Вычисляет градиент для одного наблюдения"""
    prediction = compute_logistic_prediction(features, weights)  # получаем предсказание
    error = prediction - true_label  # вычисляем ошибку
    
    gradient = [error]  # градиент для свободного члена α
    for feature in features:  # для каждого признака
        gradient.append(error * feature)  # градиент для веса признака
    
    return gradient  # возвращаем градиент

def train_logistic_regression(X_train, y_train, learning_rate=0.01, iters=1000):
    """Обучает логистическую регрессию с помощью градиентного спуска"""
    num_features = len(X_train[0])  # количество признаков
    weights = [0.0] * (num_features + 1)  # инициализируем веса (включая свободный член, поэтому +1)
    
    for iter in range(iters):  # для каждой итерации
        total_gradient = [0.0] * len(weights)  # инициализируем список для накопления градиентов
        
        for i in range(len(X_train)):  # для каждого обучающего примера (то есть пациента)
            sample_gradient = compute_gradient(X_train[i], y_train[i], weights)  # вычисляем градиент (для одного пациента)
            for j in range(len(weights)):  # для каждого веса
                total_gradient[j] += sample_gradient[j]  # суммируем градиенты
        
        # Усредняем градиент и обновляем веса
        for j in range(len(weights)):  # для каждого веса
            weights[j] -= learning_rate * (total_gradient[j] / len(X_train))  # градиентный спуск
        
        # Выводим прогресс каждые 100 итераций
        if (iter + 1) % 100 == 0:  # каждые 100 итераций
            total_loss = 0.0  # инициализируем суммарные потери
            for i in range(len(X_train)):  # для каждого обучающего примера
                total_loss += compute_log_loss(X_train[i], y_train[i], weights)  # вычисляем потери
            avg_loss = total_loss / len(X_train)  # усредняем потери
            print(f"Эпоха {iter + 1}, Средние потери: {avg_loss:.4f}")  # выводим прогресс
    
    return weights  # возвращаем обученные веса

print("Начинаем обучение логистической регрессии...")  # сообщение о начале обучения
lr_weights = train_logistic_regression(X_train, y_train, learning_rate=0.1, iters=1000)  # обучаем модель
print("Обучение завершено!")  # сообщение о завершении обучения
print(f"Обученные веса: {[f'{w:.4f}' for w in lr_weights]}")  # выводим обученные веса

Начинаем обучение логистической регрессии...
Эпоха 100, Средние потери: 0.3675
Эпоха 200, Средние потери: 0.3624
Эпоха 300, Средние потери: 0.3616
Эпоха 400, Средние потери: 0.3615
Эпоха 500, Средние потери: 0.3615
Эпоха 600, Средние потери: 0.3614
Эпоха 700, Средние потери: 0.3614
Эпоха 800, Средние потери: 0.3614
Эпоха 900, Средние потери: 0.3614
Эпоха 1000, Средние потери: 0.3614
Обучение завершено!
Обученные веса: ['0.4390', '0.1179', '0.4583', '0.7494', '0.0411', '-0.2434', '0.4525', '0.0700', '-0.3227', '0.5986', '0.3429', '0.9204']


#### ЗАДАНИЕ 4: ПРЕДСКАЗАНИЯ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

In [78]:
# 4. Напишите функцию, которая предскажет метку на тестовом экземпляре данных
def predict_logistic_regression(features, weights, threshold=0.5):
    """Предсказывает метку класса с помощью логистической регрессии"""
    probability = compute_logistic_prediction(features, weights)  # вычисляем вероятность P(y=1|x)
    if probability >= threshold:  # если вероятность >= порога (0.5 как в методичке)
        return 1  # предсказываем класс 1
    else:
        return 0  # предсказываем класс 0

# Предсказания на тестовой выборке 
print("Предсказания на тестовой выборке:")
test_predictions_lr = []  # список для предсказаний
test_probabilities_lr = []  # список для вероятностей

for i in range(len(X_test)):  # для каждого тестового примера
    features = X_test[i]  # признаки тестового примера
    true_label = y_test[i]  # истинная метка
    
    probability = compute_logistic_prediction(features, lr_weights)  # вычисляем вероятность
    predicted_label = predict_logistic_regression(features, lr_weights)  # предсказываем метку
    
    test_probabilities_lr.append(probability)  # сохраняем вероятность
    test_predictions_lr.append(predicted_label)  # сохраняем предсказание
    
    if i < 5:  # выводим первые 5 примеров для демонстрации
        print(f"Пример {i+1}: Истинная метка={true_label}, Предсказание={predicted_label}, Вероятность={probability:.4f}")

# Подсчет точности 
correct_predictions = 0  # счетчик правильных предсказаний
true_positives = 0  # истинные положительные (TP)
false_positives = 0  # ложные положительные (FP)
true_negatives = 0  # истинные отрицательные (TN)  
false_negatives = 0  # ложные отрицательные (FN)

for i in range(len(y_test)):  # для всех тестовых примеров
    true_label = y_test[i]  # истинная метка
    predicted_label = test_predictions_lr[i]  # предсказанная метка
    
    if true_label == 1 and predicted_label == 1:  # TP
        true_positives += 1
        correct_predictions += 1
    elif true_label == 1 and predicted_label == 0:  # FN
        false_negatives += 1
    elif true_label == 0 and predicted_label == 1:  # FP
        false_positives += 1
    elif true_label == 0 and predicted_label == 0:  # TN
        true_negatives += 1
        correct_predictions += 1

accuracy = correct_predictions / len(y_test)  # вычисляем точность
print(f"\nТочность логистической регрессии: {accuracy:.4f} ({correct_predictions}/{len(y_test)})")
print(f"Матрица ошибок: TP={true_positives}, FP={false_positives}, TN={true_negatives}, FN={false_negatives}")

Предсказания на тестовой выборке:
Пример 1: Истинная метка=1, Предсказание=1, Вероятность=0.9629
Пример 2: Истинная метка=1, Предсказание=1, Вероятность=0.5359
Пример 3: Истинная метка=0, Предсказание=1, Вероятность=0.9796
Пример 4: Истинная метка=1, Предсказание=1, Вероятность=0.6077
Пример 5: Истинная метка=1, Предсказание=1, Вероятность=0.9342

Точность логистической регрессии: 0.8509 (234/275)
Матрица ошибок: TP=138, FP=27, TN=96, FN=14


#### ЗАДАНИЕ 5: ВЫДЕЛЕНИЕ НЕСТАБИЛЬНЫХ СЛУЧАЕВ

In [79]:
# 5. Выделение нестабильных предсказаний
import copy  # для создания копий данных

# Создаем объединенный датасет 
dataset_with_labels = []  # список для данных с признаками и метками
for i in range(len(X_train)):  # проходим по всем обучающим примерам
    patient = X_train[i] + [y_train[i]]  # объединяем признаки и метку в один список
    dataset_with_labels.append(patient)  # добавляем пациента в общий датасет
for i in range(len(X_test)):  # проходим по всем тестовым примерам
    patient = X_test[i] + [y_test[i]]    # объединяем признаки и метку в один список  
    dataset_with_labels.append(patient)  # добавляем пациента в общий датасет

# Создаем копию данных для модификации, чтобы не испортить оригинал
dataset_copy = copy.deepcopy(dataset_with_labels)

# Порог для нестабильных предсказаний
border = 0.15  # если предсказание отличается от истины на этот порог - считаем нестабильным

# Помечаем нестабильные предсказания
for i in range(len(dataset_copy)):  # проходим по всем пациентам
    features = dataset_copy[i][:-1]  # берем все признаки (все кроме последнего элемента)
    true_label = dataset_copy[i][-1] # последний элемент - истинная метка класса
    
    prediction = compute_logistic_prediction(features, lr_weights)  # получаем предсказание модели
    
    if true_label == 0 and prediction >= border:  # если пациент здоров, но модель предсказывает болезнь
        dataset_copy[i][-1] = 2  # помечаем как нестабильный (класс 2)
    elif true_label == 1 and prediction < border:  # если пациент болен, но модель предсказывает здоровье
        dataset_copy[i][-1] = 2  # помечаем как нестабильный (класс 2)

# Подсчет и вывод результатов
class_count = [0, 0, 0] 
for patient in dataset_copy:  # проходим по всем пациентам
    class_count[patient[-1]] += 1  # увеличиваем счетчик соответствующего класса

print("Распределение классов:")
print(f"Класс 0: {class_count[0]}")  
print(f"Класс 1: {class_count[1]}") 
print(f"Класс 2: {class_count[2]}")  
print(f"Нестабильных: {class_count[2]}/{len(dataset_copy)} ({(class_count[2]/len(dataset_copy))*100:.1f}%)")  # процент нестабильных

# СОЗДАЕМ ПЕРЕМЕННЫЕ ДЛЯ ТЕРНАРНОЙ КЛАССИФИКАЦИИ
X_ternary = []  # список для признаков (все классы 0,1,2)
y_ternary = []  # список для меток (0,1,2)

for patient in dataset_copy:
    features = patient[:-1]  # все кроме последнего - признаки
    label = patient[-1]      # последний - метка класса
    X_ternary.append(features)
    y_ternary.append(label)

print(f"\nДанные для тернарной классификации подготовлены:")
print(f"Всего пациентов: {len(X_ternary)}")
print(f"Признаков на пациента: {len(X_ternary[0])}")

Распределение классов:
Класс 0: 216
Класс 1: 497
Класс 2: 205
Нестабильных: 205/918 (22.3%)

Данные для тернарной классификации подготовлены:
Всего пациентов: 918
Признаков на пациента: 11


#### ЗАДАНИЕ 6: РЕАЛИЗАЦИЯ МЕТОДА БЛИЖАЙШИХ СОСЕДЕЙ

**Метод k ближайших соседей подразумевает такую процедуру определения метки:**
1. Найти k экземпляров данных, ближайших к пробному вектору
2. Определить какая метка наиболее распространена среди этих k векторов данных
3. Присвоить эту метку пробному вектору.

In [80]:
# 6. Постройте классификацию методом ближайших соседей (БС)
def euclidean_distance(point1, point2):
    """Вычисляет евклидово расстояние между двумя точками"""
    # Формула: √(Σ(xᵢ - yᵢ)²) 
    squared_distance = 0.0  # сумма квадратов разностей
    for i in range(len(point1)):  # для каждой координаты
        squared_distance += (point1[i] - point2[i]) ** 2  # квадрат разности
    return math.sqrt(squared_distance)  # квадратный корень из суммы

def majority_vote(labels):
    """Голосование большинством с разрешением ничьих"""
    # Подсчитываем голоса вручную
    vote_counts = {}  # словарь для подсчета голосов
    for label in labels:  # для каждой метки
        if label in vote_counts:  # если метка уже есть в словаре
            vote_counts[label] += 1  # увеличиваем счетчик
        else:  # если метка новая
            vote_counts[label] = 1  # инициализируем счетчик
    
    # Находим победителя
    winner = None  # победитель
    winner_count = -1  # максимальное количество голосов
    for label, count in vote_counts.items():  # для каждой пары (метка, количество)
        if count > winner_count:  # если нашли больше голосов
            winner = label  # обновляем победителя
            winner_count = count  # обновляем максимальное количество
    
    # Проверяем ничью 
    num_winners = 0  # количество победителей
    for count in vote_counts.values():  # для каждого количества голосов, возвращает значения словаря в count
        if count == winner_count:  # если равно максимальному количеству голосов
            num_winners += 1  # увеличиваем счетчик победителей
    
    if num_winners == 1:  # если один победитель
        return winner  # возвращаем победителя
    else:  # если ничья
        return majority_vote(labels[:-1])  # рекурсивно вызываем без последнего элемента

def knn_classify(k, labeled_points, new_point):
    """Классификатор k ближайших соседей"""
    # Сортируем по расстоянию
    by_distance = []  # список кортежей для хранения (точка, метка, расстояние)
    for point, label in labeled_points:  # для каждой размеченной точки
        dist = euclidean_distance(point, new_point)  # вычисляем расстояние
        by_distance.append((point, label, dist))  # добавляем в список
    
    # Сортируем по расстоянию (от ближайшего к дальнему)
    for i in range(len(by_distance)): # проходим по всем элементам списка, i - текущая позиция, до которой уже отсортировано
        for j in range(i + 1, len(by_distance)): # начинаем со следующего элемента после i и идем до конца
            if by_distance[i][2] > by_distance[j][2]:  # если расстояние больше, меняем местами
                by_distance[i], by_distance[j] = by_distance[j], by_distance[i]

    # Берем k ближайших меток
    k_nearest_labels = []  # создаем пустой список для меток
    for i in range(k):  # проходим по первым k элементам
        label = by_distance[i][1]  # берем метку (второй элемент кортежа)
        k_nearest_labels.append(label)  # добавляем метку в список
    
    # Голосуем большинством
    return majority_vote(k_nearest_labels)  # возвращаем результат голосования

# Подготовка данных для KNN 
def prepare_knn_data(features, labels):
    """Подготавливает данные в формате (признаки, метка) для KNN"""
    return list(zip(features, labels))  # создаем список кортежей (признаки, метка)

print("Подготовка данных для KNN...")
knn_train_data = prepare_knn_data(X_train, y_train)  # готовим обучающие данные для KNN
print(f"Подготовлено {len(knn_train_data)} размеченных точек для KNN")

# Тестируем KNN с разными значениями k 
print("\nПодбор оптимального k для KNN:")
best_k = 1  # лучшее значение k
best_accuracy = 0.0  # лучшая точность

for k in range(1, 8):  # пробуем k от 1 до 7
    correct = 0  # счетчик правильных предсказаний
    for i in range(min(100, len(X_test))):  # тестируем на первых 100 примерах для скорости
        test_point = X_test[i]  # тестовая точка
        true_label = y_test[i]  # истинная метка
        
        predicted_label = knn_classify(k, knn_train_data, test_point)  # предсказываем KNN
        if predicted_label == true_label:  # если предсказание верное
            correct += 1  # увеличиваем счетчик
    
    accuracy = correct / min(100, len(X_test))  # вычисляем точность
    print(f"k={k}: Точность = {accuracy:.4f} ({correct}/{min(100, len(X_test))})")
    
    if accuracy > best_accuracy:  # если нашли лучшее k
        best_accuracy = accuracy  # обновляем лучшую точность
        best_k = k  # обновляем лучшее k

print(f"\nОптимальное k: {best_k} с точностью {best_accuracy:.4f}")

# Предсказания KNN на всей тестовой выборке с оптимальным k
print("Делаем предсказания KNN на тестовой выборке...")
knn_predictions = []  # список для предсказаний KNN
for test_point in X_test:  # для каждой тестовой точки
    predicted_label = knn_classify(best_k, knn_train_data, test_point)  # предсказываем KNN
    knn_predictions.append(predicted_label)  # сохраняем предсказание

print("Предсказания KNN завершены!")

Подготовка данных для KNN...
Подготовлено 643 размеченных точек для KNN

Подбор оптимального k для KNN:
k=1: Точность = 0.7900 (79/100)
k=2: Точность = 0.7900 (79/100)
k=3: Точность = 0.8400 (84/100)
k=4: Точность = 0.8400 (84/100)
k=5: Точность = 0.8300 (83/100)
k=6: Точность = 0.8300 (83/100)
k=7: Точность = 0.8300 (83/100)

Оптимальное k: 3 с точностью 0.8400
Делаем предсказания KNN на тестовой выборке...
Предсказания KNN завершены!


In [81]:
# Разделение данных для тернарной классификации
def split_ternary_data(features, labels, test_ratio=0.3):
    """Разделяет данные с тремя классами на обучающие и тестовые"""
    
    # Создаем списки для каждого класса
    class_0_features = []  # список для признаков класса 0
    class_0_labels = []    # список для меток класса 0
    class_1_features = []  # список для признаков класса 1
    class_1_labels = []    # список для меток класса 1
    class_2_features = []  # список для признаков класса 2
    class_2_labels = []    # список для меток класса 2
    
    # Разделяем данные по классам
    for i in range(len(features)):  # проходим по всем данным
        if labels[i] == 0:  # если метка класса 0
            class_0_features.append(features[i])  # добавляем признаки в класс 0
            class_0_labels.append(labels[i])       # добавляем метку в класс 0
        elif labels[i] == 1:  # если метка класса 1
            class_1_features.append(features[i])  # добавляем признаки в класс 1
            class_1_labels.append(labels[i])       # добавляем метку в класс 1
        else:  # если метка класса 2
            class_2_features.append(features[i])  # добавляем признаки в класс 2
            class_2_labels.append(labels[i])       # добавляем метку в класс 2
    
    # Перемешиваем каждый класс отдельно
    random.shuffle(class_0_features)  # перемешиваем признаки класса 0
    random.shuffle(class_1_features)  # перемешиваем признаки класса 1
    random.shuffle(class_2_features)  # перемешиваем признаки класса 2
    
    # Вычисляем размеры тестовых выборок для каждого класса
    test_size_0 = int(len(class_0_features) * test_ratio)  # 30% от класса 0
    test_size_1 = int(len(class_1_features) * test_ratio)  # 30% от класса 1
    test_size_2 = int(len(class_2_features) * test_ratio)  # 30% от класса 2
    
    # ФорМИРУЕМ ТЕСТОВУЮ ВЫБОРКУ
    X_test_ternary = []  # список для тестовых признаков
    y_test_ternary = []  # список для тестовых меток
    
    # Добавляем данные класса 0 в тестовую выборку
    for i in range(test_size_0):  # для первых test_size_0 элементов класса 0
        X_test_ternary.append(class_0_features[i])  # добавляем признаки в тест
        y_test_ternary.append(class_0_labels[i])     # добавляем метки в тест
    
    # Добавляем данные класса 1 в тестовую выборку
    for i in range(test_size_1):  # для первых test_size_1 элементов класса 1
        X_test_ternary.append(class_1_features[i])  # добавляем признаки в тест
        y_test_ternary.append(class_1_labels[i])     # добавляем метки в тест
    
    # Добавляем данные класса 2 в тестовую выборку
    for i in range(test_size_2):  # для первых test_size_2 элементов класса 2
        X_test_ternary.append(class_2_features[i])  # добавляем признаки в тест
        y_test_ternary.append(class_2_labels[i])     # добавляем метки в тест
    
    # ФОРМИРУЕМ ОБУЧАЮЩУЮ ВЫБОРКУ
    X_train_ternary = []  # список для обучающих признаков
    y_train_ternary = []  # список для обучающих меток
    
    # Добавляем данные класса 0 в обучающую выборку (оставшиеся после теста)
    for i in range(test_size_0, len(class_0_features)):  # от test_size_0 до конца
        X_train_ternary.append(class_0_features[i])  # добавляем признаки в обучение
        y_train_ternary.append(class_0_labels[i])     # добавляем метки в обучение
    
    # Добавляем данные класса 1 в обучающую выборку (оставшиеся после теста)
    for i in range(test_size_1, len(class_1_features)):  # от test_size_1 до конца
        X_train_ternary.append(class_1_features[i])  # добавляем признаки в обучение
        y_train_ternary.append(class_1_labels[i])     # добавляем метки в обучение
    
    # Добавляем данные класса 2 в обучающую выборку (оставшиеся после теста)
    for i in range(test_size_2, len(class_2_features)):  # от test_size_2 до конца
        X_train_ternary.append(class_2_features[i])  # добавляем признаки в обучение
        y_train_ternary.append(class_2_labels[i])     # добавляем метки в обучение
    
    # Перемешиваем обучающую выборку
    train_combined = []  # список для объединенных данных обучения
    for i in range(len(X_train_ternary)):  # проходим по всем обучающим данным
        pair = (X_train_ternary[i], y_train_ternary[i])  # создаем пару (признаки, метка)
        train_combined.append(pair)  # добавляем пару в объединенный список
    
    random.shuffle(train_combined)  # перемешиваем объединенные данные обучения
    
    # Перемешиваем тестовую выборку
    test_combined = []  # список для объединенных данных теста
    for i in range(len(X_test_ternary)):  # проходим по всем тестовым данным
        pair = (X_test_ternary[i], y_test_ternary[i])  # создаем пару (признаки, метка)
        test_combined.append(pair)  # добавляем пару в объединенный список
    
    random.shuffle(test_combined)  # перемешиваем объединенные данные теста
    
    # Разделяем объединенные данные обратно на признаки и метки
    X_train_ternary = []  # очищаем список для обучающих признаков
    y_train_ternary = []  # очищаем список для обучающих меток
    
    for item in train_combined:  # проходим по всем элементам объединенных данных обучения
        X_train_ternary.append(item[0])  # добавляем признаки (первый элемент пары)
        y_train_ternary.append(item[1])  # добавляем метки (второй элемент пары)
    
    X_test_ternary = []  # очищаем список для тестовых признаков
    y_test_ternary = []  # очищаем список для тестовых меток
    
    for item in test_combined:  # проходим по всем элементам объединенных данных теста
        X_test_ternary.append(item[0])  # добавляем признаки (первый элемент пары)
        y_test_ternary.append(item[1])  # добавляем метки (второй элемент пары)
    
    return X_train_ternary, X_test_ternary, y_train_ternary, y_test_ternary

# Разделяем данные для тернарной классификации
X_train_ternary, X_test_ternary, y_train_ternary, y_test_ternary = split_ternary_data(X_ternary, y_ternary)

# Выводим информацию о разделенных данных
print("Данные разделены для тернарной классификации:")
print(f"Обучающая выборка: {len(X_train_ternary)} пациентов")
print(f"Тестовая выборка: {len(X_test_ternary)} пациентов")

# Подсчитываем распределение классов в обучающей выборке
class_count_train = [0, 0, 0]  # счетчики для классов 0, 1, 2
for label in y_train_ternary:  # проходим по всем меткам обучения
    class_count_train[label] += 1  # увеличиваем счетчик соответствующего класса

print(f"Распределение классов в обучении: 0={class_count_train[0]}, 1={class_count_train[1]}, 2={class_count_train[2]}")

# Подсчитываем распределение классов в тестовой выборке
class_count_test = [0, 0, 0]  # счетчики для классов 0, 1, 2
for label in y_test_ternary:  # проходим по всем меткам теста
    class_count_test[label] += 1  # увеличиваем счетчик соответствующего класса

print(f"Распределение классов в тесте: 0={class_count_test[0]}, 1={class_count_test[1]}, 2={class_count_test[2]}")

Данные разделены для тернарной классификации:
Обучающая выборка: 644 пациентов
Тестовая выборка: 274 пациентов
Распределение классов в обучении: 0=152, 1=348, 2=144
Распределение классов в тесте: 0=64, 1=149, 2=61


#### ЗАДАНИЕ 7: ВЫЧИСЛЕНИЕ МЕТРИК КАЧЕСТВА КЛАССИФИКАЦИИ

In [82]:
# 7. Измерьте качество классификации по метрикам Precision, Recall, F1 (бинарная) и Accuracy (тернарная)
def calculate_binary_metrics(true_labels, predicted_labels):
    """Вычисляет метрики для бинарной классификации"""
    true_positives = 0  # истинные положительные
    false_positives = 0  # ложные положительные  
    true_negatives = 0  # истинные отрицательные
    false_negatives = 0  # ложные отрицательные
    
    # Подсчет матрицы ошибок 
    for true, pred in zip(true_labels, predicted_labels):
        if true == 1 and pred == 1:  # TP - пациент болен и мы правильно это определили
            true_positives += 1
        elif true == 1 and pred == 0:  # FN - пациент болен, но мы сказали "здоров" 
            false_negatives += 1
        elif true == 0 and pred == 1:  # FP - пациент здоров, но мы сказали "болен" 
            false_positives += 1
        elif true == 0 and pred == 0:  # TN - пациент здоров и мы правильно это определили
            true_negatives += 1
    
    # Вычисление метрик 
    # Точность: Из всех пациентов, которых мы назвали больными, сколько действительно больны?
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    # Полнота: Из всех реально больных пациентов, сколько мы правильно выявили?
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    # F1 - среднее гармоническое для точности и полноты
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    # Правильность - общая доля правильных предсказаний
    accuracy = (true_positives + true_negatives) / len(true_labels) if len(true_labels) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall, 
        'f1': f1_score,
        'accuracy': accuracy,
        'tp': true_positives,
        'fp': false_positives,
        'tn': true_negatives,
        'fn': false_negatives
    }

def calculate_ternary_accuracy(true_labels, predicted_labels):
    """Вычисляет accuracy для тернарной классификации"""
    correct = 0  # счетчик правильных предсказаний
    for true, pred in zip(true_labels, predicted_labels):  # для каждой пары
        if true == pred:  # если метки совпадают
            correct += 1  # увеличиваем счетчик
    # Accuracy = Правильные_предсказания / Всего_предсказаний
    return correct / len(true_labels) if len(true_labels) > 0 else 0 

print("=" * 60)
print("МЕТРИКИ КАЧЕСТВА КЛАССИФИКАЦИИ")
print("=" * 60)

# Метрики для логистической регрессии (бинарная)
print("\n--- ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ (БИНАРНАЯ) ---")
lr_binary_metrics = calculate_binary_metrics(y_test, test_predictions_lr)
print(f"Precision: {lr_binary_metrics['precision']:.4f}")
print(f"Recall:    {lr_binary_metrics['recall']:.4f}") 
print(f"F1-score:  {lr_binary_metrics['f1']:.4f}")
print(f"Accuracy:  {lr_binary_metrics['accuracy']:.4f}")
print(f"Матрица ошибок: TP={lr_binary_metrics['tp']}, FP={lr_binary_metrics['fp']}, TN={lr_binary_metrics['tn']}, FN={lr_binary_metrics['fn']}")

# Метрики для KNN (бинарная)
print("\n--- МЕТОД БЛИЖАЙШИХ СОСЕДЕЙ (БИНАРНАЯ) ---")
knn_binary_metrics = calculate_binary_metrics(y_test, knn_predictions)
print(f"Precision: {knn_binary_metrics['precision']:.4f}")
print(f"Recall:    {knn_binary_metrics['recall']:.4f}")
print(f"F1-score:  {knn_binary_metrics['f1']:.4f}")
print(f"Accuracy:  {knn_binary_metrics['accuracy']:.4f}")
print(f"Матрица ошибок: TP={knn_binary_metrics['tp']}, FP={knn_binary_metrics['fp']}, TN={knn_binary_metrics['tn']}, FN={knn_binary_metrics['fn']}")

# Метрики для тернарной классификации (KNN)
print("\n--- ТЕРНАРНАЯ КЛАССИФИКАЦИЯ (KNN) ---")
# Подготовка данных для тернарной KNN
knn_train_ternary = prepare_knn_data(X_train_ternary, y_train_ternary)

# Предсказания для тернарной классификации
knn_ternary_predictions = []
for test_point in X_test_ternary:
    predicted_label = knn_classify(best_k, knn_train_ternary, test_point)
    knn_ternary_predictions.append(predicted_label)

ternary_accuracy = calculate_ternary_accuracy(y_test_ternary, knn_ternary_predictions)
print(f"Accuracy: {ternary_accuracy:.4f}")

# Распределение предсказаний в тернарной классификации
ternary_distribution = {}
for label in knn_ternary_predictions:
    ternary_distribution[label] = ternary_distribution.get(label, 0) + 1

print("Распределение предсказанных классов:")
for label, count in sorted(ternary_distribution.items()):
    percentage = (count / len(knn_ternary_predictions)) * 100
    print(f"  Класс {label}: {count} ({percentage:.1f}%)")

МЕТРИКИ КАЧЕСТВА КЛАССИФИКАЦИИ

--- ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ (БИНАРНАЯ) ---
Precision: 0.8364
Recall:    0.9079
F1-score:  0.8707
Accuracy:  0.8509
Матрица ошибок: TP=138, FP=27, TN=96, FN=14

--- МЕТОД БЛИЖАЙШИХ СОСЕДЕЙ (БИНАРНАЯ) ---
Precision: 0.8333
Recall:    0.9211
F1-score:  0.8750
Accuracy:  0.8545
Матрица ошибок: TP=140, FP=28, TN=95, FN=12

--- ТЕРНАРНАЯ КЛАССИФИКАЦИЯ (KNN) ---
Accuracy: 0.8212
Распределение предсказанных классов:
  Класс 0: 71 (25.9%)
  Класс 1: 169 (61.7%)
  Класс 2: 34 (12.4%)


#### ЗАДАНИЕ 8: ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ И ВЫВОДЫ

In [83]:
# 8. Интерпретация результатов - выводы о качестве классификации
print("=" * 60)
print("ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ")
print("=" * 60)

print("\n1. СРАВНЕНИЕ МЕТОДОВ КЛАССИФИКАЦИИ:")
print(f"   • Логистическая регрессия: Accuracy = {lr_binary_metrics['accuracy']:.4f}")
print(f"   • Метод ближайших соседей: Accuracy = {knn_binary_metrics['accuracy']:.4f}")

if lr_binary_metrics['accuracy'] > knn_binary_metrics['accuracy']:
    print("   ✓ Логистическая регрессия показала лучшую точность")
elif knn_binary_metrics['accuracy'] > lr_binary_metrics['accuracy']:
    print("   ✓ Метод ближайших соседей показал лучшую точность")
else:
    print("   ○ Оба метода показали одинаковую точность")

print("\n2. АНАЛИЗ НЕСТАБИЛЬНЫХ ПРЕДСКАЗАНИЙ:")
unstable_count = sum(1 for label in y_test_ternary if label == 2)
unstable_percentage = (unstable_count / len(y_test_ternary)) * 100
print(f"   • Нестабильных случаев в тестовой выборке: {unstable_count} ({unstable_percentage:.1f}%)")
print("   • Эти случаи требуют дополнительного внимания врача")

print("\n3. КАЧЕСТВО ТЕРНАРНОЙ КЛАССИФИКАЦИИ:")
print(f"   • Accuracy тернарной классификации: {ternary_accuracy:.4f}")
print("   • Тернарная классификация позволяет выделить 'сомнительные' случаи")

print("\n4. ВЫВОДЫ:")
print("   • Оба метода демонстрируют хорошее качество классификации")
print("   • Логистическая регрессия более интерпретируема (можно анализировать веса)")
print("   • KNN не требует обучения, но медленнее на этапе предсказания")
print("   • Введение третьего класса повышает надежность медицинской системы")
print("   • Для критических решений важна не только точность, но и возможность")
print("     выделения случаев, требующих дополнительной проверки")

ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ

1. СРАВНЕНИЕ МЕТОДОВ КЛАССИФИКАЦИИ:
   • Логистическая регрессия: Accuracy = 0.8509
   • Метод ближайших соседей: Accuracy = 0.8545
   ✓ Метод ближайших соседей показал лучшую точность

2. АНАЛИЗ НЕСТАБИЛЬНЫХ ПРЕДСКАЗАНИЙ:
   • Нестабильных случаев в тестовой выборке: 61 (22.3%)
   • Эти случаи требуют дополнительного внимания врача

3. КАЧЕСТВО ТЕРНАРНОЙ КЛАССИФИКАЦИИ:
   • Accuracy тернарной классификации: 0.8212
   • Тернарная классификация позволяет выделить 'сомнительные' случаи

4. ВЫВОДЫ:
   • Оба метода демонстрируют хорошее качество классификации
   • Логистическая регрессия более интерпретируема (можно анализировать веса)
   • KNN не требует обучения, но медленнее на этапе предсказания
   • Введение третьего класса повышает надежность медицинской системы
   • Для критических решений важна не только точность, но и возможность
     выделения случаев, требующих дополнительной проверки
